In [35]:
%pip install pyspark==3.5.1
%pip install pandas
%pip install findspark
%pip install joblib
%pip install scikit-learn
%pip install xgboost
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 33.3 MB/s  0:00:01m0:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
# PATH definitions
DATA_DIR = "./dataset"  # đổi nếu dùng Drive
MODEL_DIR = "./models"
TRAIN_PATH = os.path.join(DATA_DIR, "train_data.csv")
VAL_PATH   = os.path.join(DATA_DIR, "val_data.csv")
TEST_PATH  = os.path.join(DATA_DIR, "test_data.csv")

In [3]:
# Columns
TEXT_COL = "Review"
ASPECTS  = ["Price","Shipping","Outlook","Quality","Size","Shop_Service","General","Others"]

# Label mapping
SENT_ID2NAME = {-1: "None", 0: "Negative", 1: "Positive", 2: "Neutral"}
LABEL_VALUES = list(SENT_ID2NAME.keys())  # [-1, 0, 1, 2]
LABEL_NAMES  = list(SENT_ID2NAME.values())  # ["None","Negative","Positive","Neutral"]

In [1]:
# Utilities function 
import re, unicodedata
import pandas as pd

URL_RE = re.compile(r"https?://\S+|www\.\S+")
TAG_RE = re.compile(r"<[^>]+>")
MULTISPACE_RE = re.compile(r"\s+")
VIETNAMESE_BASIC_STOPWORDS = set("""
và hoặc nhưng là thì mà được bị của cho với về từ tới đến nỗi do vì nên nếu khi để bằng như lại đã đang sẽ không chưa chẳng rất quá lắm hơi
này kia nọ đó đây ấy vậy thế sao tại vì do đó tuy nhiên hơn kém chỉ mỗi một các những cái con chiếc đôi đc nhé nha ạ ơi
""".split())

def preprocess_xgb(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.strip().lower()
    text = URL_RE.sub(" ", text)
    text = TAG_RE.sub(" ", text)
    text = text.replace("❤️", " yeu ").replace("❤", " yeu ").replace("😍", " yeu ")
    text = re.sub(r"[^\w\sáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ]", " ", text)
    text = unicodedata.normalize("NFC", text)  # normalize accents
    text = MULTISPACE_RE.sub(" ", text).strip()
    tokens = [w for w in text.split() if w not in VIETNAMESE_BASIC_STOPWORDS]
    return " ".join(tokens)

In [4]:
from pyspark.sql import SparkSession
import findspark
from pyspark.sql import SparkSession

try:
    spark.stop()
except:
    pass

scala_version = '2.12'
spark_version = '3.5.1'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' , 'org.apache.kafka:kafka-clients:3.5.1' ]

findspark.init()
spark = SparkSession.builder.master("local").appName("kafka").config("spark.jars.packages", ",".join(packages)).getOrCreate()


spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
25/10/17 19:11:12 WARN Utils: Your hostname, DESKTOP-1JUULPQ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/17 19:11:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kaos/.ivy2/cache
The jars for the packages stored in: /home/kaos/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-96493e85-5f80-485a-8d13-bdceef09c17e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.5.1 in central
	found com.github.luben#zstd-jni;1.5.5-1 in central
	f

In [22]:
topic_name = 'ABSA'
kafka_server = 'localhost:9092'

df_raw = spark.readStream.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).load()

In [6]:
df_raw .printSchema()
df_raw.toPandas()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,0,2025-10-17 19:10:37.991,0
1,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,1,2025-10-17 19:10:42.989,0
2,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,2,2025-10-17 19:10:48.724,0
3,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,3,2025-10-17 19:10:53.725,0
4,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,4,2025-10-17 19:10:58.733,0
5,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,5,2025-10-17 19:11:03.734,0
6,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,6,2025-10-17 19:11:08.735,0
7,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,7,2025-10-17 19:11:13.733,0
8,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,8,2025-10-17 19:11:18.734,0
9,None,"[123, 34, 82, 101, 118, 105, 101, 119, 34, 58,...",ABSA,0,9,2025-10-17 19:11:23.736,0


In [7]:
from time import sleep
from IPython.display import display, clear_output
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

input_schema = StructType([
    StructField("Review", StringType()),
])

df_input = df_raw.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), input_schema).alias("data")) \
    .select("data.*")


for i in range(0, 100):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {i*5}")
        display(df_input.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break

print("Live view ended...")

Showing live view refreshed every 5 seconds
Seconds passed: 0


,Review
0,Hàng đẹp so với giá nha .nhưng đóng gói không ...
1,Dịch vụ khách hàng giới thiệu sản phẩm rất tốt...
2,Hàng đẹp so với giá nha .nhưng đóng gói không ...
3,Dịch vụ khách hàng giới thiệu sản phẩm rất tốt...
4,giao nhanh giày đẹp êm chân nhưng from giày hơ...
5,Ngoài ngắm cực phẩm thi đấu trong sân thì ngoà...
6,Dài xinh nha from lên đẹp xỉu êm chân lắm nha ...
7,"Giao hàng nhanh Có Giày khá cứng, lót giày thừ..."
8,"Giầy đi êm, giá cả hợp lý, đóng gói cẩn thận, ..."
9,"Dày giá rẻ nên đừng mong đợi j nhiều, tiền nào..."


break
Live view ended...


In [17]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
import numpy as np
import joblib

# Pre-load models/encoders and broadcast them
bc_models = {
    aspect: spark.sparkContext.broadcast(joblib.load(os.path.join(MODEL_DIR, f"{aspect}_xgb.pkl")))
    for aspect in ASPECTS
}
bc_encoders = {
    aspect: spark.sparkContext.broadcast(joblib.load(os.path.join(MODEL_DIR, f"{aspect}_encoder.pkl")))
    for aspect in ASPECTS
}


# ----------------------
# UDF factory for each aspect
# ----------------------
def make_predict_udf(aspect):
    model = bc_models[aspect].value
    encoder = bc_encoders[aspect].value

    @pandas_udf(StringType())
    def predict_udf(texts: pd.Series) -> pd.Series:
        texts_proc = [preprocess_xgb(t) for t in texts]
        preds_encoded = model.predict(texts_proc)
        preds_original = encoder.inverse_transform(preds_encoded)
        return pd.Series([SENT_ID2NAME[int(x)] for x in preds_original])

    return predict_udf

In [23]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, StructField

input_schema = StructType([
    StructField("Review", StringType()),
])

df_input = (
    df_raw
    .selectExpr("CAST(value AS STRING) as json")
    .select(from_json(col("json"), input_schema).alias("data"))
    .select("data.*")
)

# Apply UDFs
for aspect in ASPECTS:
    predict_udf = make_predict_udf(aspect)
    df_input = df_input.withColumn(f"{aspect}_pred", predict_udf("Review"))

cols_to_show = ["Review"] + [f"{aspect}_pred" for aspect in ASPECTS]

# Define per-batch function for live console output
def foreach_batch(df, batch_id):
    print(f"\n=== Batch {batch_id} ===")
    df.select(*cols_to_show).show(10, truncate=False)

# WriteStream triggers every 5 seconds
query = (
    df_input.writeStream
    .foreachBatch(foreach_batch)
    .trigger(processingTime="5 seconds")
    .start()
)

query.awaitTermination()



=== Batch 0 ===
+------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Review|Price_pred|Shipping_pred|Outlook_pred|Quality_pred|Size_pred|Shop_Service_pred|General_pred|Others_pred|
+------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
+------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+


=== Batch 1 ===


+---------------------------------------------------------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Review                                                                                                         |Price_pred|Shipping_pred|Outlook_pred|Quality_pred|Size_pred|Shop_Service_pred|General_pred|Others_pred|
+---------------------------------------------------------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Đẹp lắm mng oiii Đế hơi cứng đi vài lần sẽ mềm mng yên tâm nha Shop tư vấn rất nhiệt tình  Sẽ ủng hộ shop nhiều|None      |None         |Positive    |None        |None     |None             |None        |None       |
+---------------------------------------------------------------------------------------------------------------+----------+----

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Review                                                                                                                                                                                                                                            |Price_pred|Shipping_pred|Outlook_pred|Quality_pred|Size_pred|Shop_Service_pred|General_pred|Others_pred|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-------------+------------+------------+-------

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Review                                                                                                                                                                                                                                                                                 |Price_pred|Shipping_pred|Outlook_pred|Quality_pred|Size_pred|Shop_Service_pred|General_pred|Others_pred|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

+----------------------------------------------------------------------------------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Review                                                                                                                                  |Price_pred|Shipping_pred|Outlook_pred|Quality_pred|Size_pred|Shop_Service_pred|General_pred|Others_pred|
+----------------------------------------------------------------------------------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|shop đóng gói cẩn thận , đúng với mô tả , tuy có một vài chỗ ko đc hoàn mỹ nhưng shop làm việc uy tín nên mik bỏ qua , nên mua nha mn 5⭐|None      |None         |None        |Neutral     |None     |Positive         |None        |None       |
+---------------------------

...

+-----------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Review                                                           |Price_pred|Shipping_pred|Outlook_pred|Quality_pred|Size_pred|Shop_Service_pred|General_pred|Others_pred|
+-----------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Tui lấy size 41 hơi chật chân nên 4 sao thôi với lại giầy hơi lỗi|None      |None         |None        |Negative    |Negative |None             |None        |None       |
+-----------------------------------------------------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 


=== Batch 41 ===


25/10/17 19:31:30 ERROR Executor: Exception in task 0.0 in stage 53.0 (TID 53)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1094, in main
    split_index = read_int(infile)
                  ^^^^^^^^^^^^^^^^
  File "/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 594, in read_int
    length = stream.read(4)
             ^^^^^^^^^^^^^^
KeyboardInterrupt

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:118)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala